# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
from sklearn import datasets, model_selection, metrics, ensemble, preprocessing, tree
import numpy as np

raw_data = datasets.load_boston()
boston_data = raw_data.data
boston_target = raw_data.target
scalar = preprocessing.StandardScaler()
train_data = boston_data[:-int(boston_data.shape[0] / 4),:]
test_data = boston_data[-int(boston_data.shape[0] / 4):, :]
train_target = boston_target[:-int(boston_data.shape[0] / 4)]
test_target = boston_target[-int(boston_data.shape[0] / 4):]
print('SHAPE TARGET REAL:', boston_target.shape, 'SHAPE NEW:', train_target.shape, test_target.shape)
print('SHAPE DATA REAL:', boston_data.shape, 'SHAPE NEW:', train_data.shape, test_data.shape)

SHAPE TARGET REAL: (506,) SHAPE NEW: (380,) (126,)
SHAPE DATA REAL: (506, 13) SHAPE NEW: (380, 13) (126, 13)


In [2]:
def write_answer_1(ans, number):
    with open("answer_task_2_number_"+str(number)+".txt", "w") as text_file:
        text_file.write(str(ans))

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [3]:
def vector_s_finder(composition, real_target, x):
    s = []
    for i in range(len(real_target)):
        s.append(-1*(np.dot(composition, x) - real_target))
    return s

composition = [1, 1, 1, 2]
real_target = [0, 0, 0, 1]
x = [1, 1, 1, 1]

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [4]:
def s_finder(composition, y_real):
    s_vector = []
    for _ in range(len(y_real)):
        s_vector.append(-1 * (composition[_] - y_real[_]))
    return s_vector

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(estimators_array, estimators_coffs_array)]) for x in X]
n_iterations = 50
estimators_array = []
estimators_coffs_array = []
target = train_target
for i in range(n_iterations):
    estimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    estimator.fit(train_data, target)
    estimators_array.append(estimator)
    estimators_coffs_array.append(.9)
    compose_predict = gbm_predict(train_data)
    vectos_s = s_finder(compose_predict, train_target)
#    print(vectos_s)
    target = vectos_s

In [5]:
print('-----')
error = metrics.mean_squared_error(test_target, gbm_predict(test_data))
print('Mean_Squared_Error:', error)
print('----------')
print('RMSE:', np.sqrt(error))
write_answer_1(np.sqrt(error), 2)

-----
Mean_Squared_Error: 29.759444504536617
----------
RMSE: 5.455221764927308


In [6]:
'''
estimators_array = []
estimators_coffs_array = []
target = train_target
estimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
estimator.fit(train_data, target)
estimators_array.append(estimator)
estimators_coffs_array.append(.9)
compose_predict = gbm_predict(train_data)
print(len(compose_predict))
target = s_finder(compose_predict, train_target)
print(np.asmatrix(target).shape)
estimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
estimator.fit(train_data, target)
estimators_array.append(estimator)
estimators_coffs_array.append(.9)
compose_predict = gbm_predict(train_data)
print(compose_predict)
'''

'\nestimators_array = []\nestimators_coffs_array = []\ntarget = train_target\nestimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)\nestimator.fit(train_data, target)\nestimators_array.append(estimator)\nestimators_coffs_array.append(.9)\ncompose_predict = gbm_predict(train_data)\nprint(len(compose_predict))\ntarget = s_finder(compose_predict, train_target)\nprint(np.asmatrix(target).shape)\nestimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)\nestimator.fit(train_data, target)\nestimators_array.append(estimator)\nestimators_coffs_array.append(.9)\ncompose_predict = gbm_predict(train_data)\nprint(compose_predict)\n'

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [7]:
def s_finder(composition, y_real):
    s_vector = []
    for _ in range(len(y_real)):
        s_vector.append(-1 * (composition[_] - y_real[_]))
    return s_vector

def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(estimators_array, estimators_coffs_array)]) for x in X]
n_iterations = 50
estimators_array = []
estimators_coffs_array = []
target = train_target
for i in range(n_iterations):
    estimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
    estimator.fit(train_data, target)
    estimators_array.append(estimator)
    estimators_coffs_array.append(.9 / (1 + i))
    compose_predict = gbm_predict(train_data)
    vectos_s = s_finder(compose_predict, train_target)
#    print(vectos_s)
    target = vectos_s

In [8]:
print('-----')
error = metrics.mean_squared_error(test_target, gbm_predict(test_data))
print('Mean_Squared_Error:', error)
print('----------')
print('RMSE:', np.sqrt(error))
write_answer_1(np.sqrt(error), 3)


-----
Mean_Squared_Error: 27.46843303673391
----------
RMSE: 5.241033584774468


## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [9]:
'''
import matplotlib.pylab as plt
### ITERATIONS
iterations = [2, 5, 8, 12, 14, 16, 18, 20, 25]
for n_iterations in iterations:
    def s_finder(composition, y_real):
        s_vector = []
        for _ in range(len(y_real)):
            s_vector.append(-1 * (composition[_] - y_real[_]))
        return s_vector

    def gbm_predict(X):
        return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(estimators_array, estimators_coffs_array)]) for x in X]
    n_iterations = n_iterations
    estimators_array = []
    estimators_coffs_array = []
    target = train_target
    for i in range(n_iterations):
        estimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)
        estimator.fit(train_data, target)
        estimators_array.append(estimator)
        estimators_coffs_array.append(.9 / (1 + i))
        compose_predict = gbm_predict(train_data)
        vectos_s = s_finder(compose_predict, train_target)
    #    print(vectos_s)
        target = vectos_s
    print('-----')
    error = metrics.mean_squared_error(test_target, gbm_predict(test_data))
    print('Mean_Squared_Error for iterations',str(n_iterations),':', error)
    print('----------')
    print('RMSE for iterations', str(n_iterations),':', np.sqrt(error))
    plt.plot(n_iterations, np.sqrt(error), 'b-', marker='o')
    plt.xlabel('Количество итераций')
    plt.ylabel('Значение RMSE')
    plt.title('Изменение кол-ва итераций')
plt.show()
#Глубина деревьев
trees_depth = [1, 3, 5, 10, 15, 30, 50, 100, 150, 200, 300]
for depth in trees_depth:
    def s_finder(composition, y_real):
        s_vector = []
        for _ in range(len(y_real)):
            s_vector.append(-1 * (composition[_] - y_real[_]))
        return s_vector

    def gbm_predict(X):
        return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(estimators_array, estimators_coffs_array)]) for x in X]
    n_iterations = 20
    estimators_array = []
    estimators_coffs_array = []
    target = train_target
    for i in range(n_iterations):
        estimator = tree.DecisionTreeRegressor(max_depth=depth, random_state=42)
        estimator.fit(train_data, target)
        estimators_array.append(estimator)
        estimators_coffs_array.append(.9 / (1 + i))
        compose_predict = gbm_predict(train_data)
        vectos_s = s_finder(compose_predict, train_target)
    #    print(vectos_s)
        target = vectos_s
    print('-----')
    error = metrics.mean_squared_error(test_target, gbm_predict(test_data))
    print('Mean_Squared_Error for depth',str(depth),':', error)
    print('----------')
    print('RMSE for depth', str(depth),':', np.sqrt(error))
    plt.plot(depth, np.sqrt(error), 'b-', marker='o')
    plt.xlabel('Глубина дерева')
    plt.ylabel('Значение RMSE')
    plt.title('Изменение глубины деревьев')
plt.show()
'''

"\nimport matplotlib.pylab as plt\n### ITERATIONS\niterations = [2, 5, 8, 12, 14, 16, 18, 20, 25]\nfor n_iterations in iterations:\n    def s_finder(composition, y_real):\n        s_vector = []\n        for _ in range(len(y_real)):\n            s_vector.append(-1 * (composition[_] - y_real[_]))\n        return s_vector\n\n    def gbm_predict(X):\n        return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(estimators_array, estimators_coffs_array)]) for x in X]\n    n_iterations = n_iterations\n    estimators_array = []\n    estimators_coffs_array = []\n    target = train_target\n    for i in range(n_iterations):\n        estimator = tree.DecisionTreeRegressor(max_depth=5, random_state=42)\n        estimator.fit(train_data, target)\n        estimators_array.append(estimator)\n        estimators_coffs_array.append(.9 / (1 + i))\n        compose_predict = gbm_predict(train_data)\n        vectos_s = s_finder(compose_predict, train_target)\n    #    print(vectos_s)\n        tar

# Ответ на задание 4
# 1 False
# 2 True
# 3 False
# True

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [10]:
from sklearn import linear_model
linear_reg = linear_model.LinearRegression()
linear_reg.fit(train_data, train_target)
error = metrics.mean_squared_error(test_target, linear_reg.predict(test_data))
print('TASK 5 RMSE = ', np.sqrt(error))
write_answer_1(np.sqrt(error), 5)

TASK 5 RMSE =  7.819688142087423
